In [1]:
import pandas as pd
import os

In [36]:
base_path = os.getcwd()
base_path

'C:\\Users\\ragor\\OneDrive\\Dokumente\\Bachelorarbeit\\BA-Prediction-Beyond-the-Systems-Latency\\01_Data Collection\\02_Data\\MT Analysis\\demographic'

# Load Data

## Create Dataframe for each MT csv

In [2]:
all_data = []
if os.getcwd().split("\\")[len(os.getcwd().split("\\")) - 1] != "mt_data":
    os.chdir('./mt_data')

for f in os.listdir():
    part_id = f.split("_")[1].split(".")[0]

    curr = pd.read_csv(f, sep=";")
    curr["part_id"] = part_id

    curr = curr[['part_id', 'width', 'length', 'MT']]

    all_data.append(curr)

print(len(all_data))
all_data[0].head()

60


,part_id,width,length,MT
0,12299398,57,1207,-1
1,12299398,57,1207,-1
2,12299398,57,1207,7171
3,12299398,34,905,12359
4,12299398,23,302,-1


# Add demographic data

In [3]:
if os.getcwd().split("\\")[len(os.getcwd().split("\\")) - 1] != "demographic":
    os.chdir('../demographic')

for f in os.listdir():
    curr_id = f.split("_")[1].split(".")[0]
    for i, datapoint in enumerate(all_data):
        if datapoint['part_id'][0] == curr_id:
            demographic_data = pd.read_csv(f, sep=';')
            all_data[i] = pd.concat([datapoint, demographic_data], axis=1)

#print(all_data[0].head())

for datapoint in all_data:
    # print(datapoint['Age'].values[0])
    datapoint['Age'] = datapoint['Age'].values[0]
    datapoint['Gender'] = datapoint['Gender'].values[0]
    datapoint.drop('Screen Resolution', inplace=True, axis=1)
    datapoint['Screen Size in inch'] = datapoint['Screen Size in inch'].values[0]
    datapoint.drop('Screen Refresh Rate', inplace=True, axis=1)
    datapoint.drop('OS', inplace=True,axis=1)
    datapoint.drop('Inputdevice', inplace=True,axis=1)
    datapoint['Real Res'] = datapoint['Real Res'].values[0]

all_data[0].head()

,part_id,width,length,MT,Age,Gender,Screen Size in inch,Real Res
0,12299398,57,1207,-1,25.0,male,14.0,1366 x 768px
1,12299398,57,1207,-1,25.0,male,14.0,1366 x 768px
2,12299398,57,1207,7171,25.0,male,14.0,1366 x 768px
3,12299398,34,905,12359,25.0,male,14.0,1366 x 768px
4,12299398,23,302,-1,25.0,male,14.0,1366 x 768px


## Add ID column

In [4]:
for datapoint in all_data:
    datapoint['ID'] = datapoint['length'] / datapoint['width']

sorted_data = []
for datapoint in all_data:
    curr = datapoint[['part_id', 'width', 'length', 'ID', 'MT', 'Age', 'Gender', 'Screen Size in inch', 'Real Res']]
    sorted_data.append(curr)

all_data = sorted_data
all_data[0]

,part_id,width,length,ID,MT,Age,Gender,Screen Size in inch,Real Res
0,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px
1,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px
2,12299398,57,1207,21.175439,7171,25.0,male,14.0,1366 x 768px
3,12299398,34,905,26.617647,12359,25.0,male,14.0,1366 x 768px
4,12299398,23,302,13.130435,-1,25.0,male,14.0,1366 x 768px
...,...,...,...,...,...,...,...,...,...
202,12299398,102,905,8.872549,768,25.0,male,14.0,1366 x 768px
203,12299398,68,1207,17.750000,1834,25.0,male,14.0,1366 x 768px
204,12299398,90,1207,13.411111,1468,25.0,male,14.0,1366 x 768px
205,12299398,90,302,3.355556,255,25.0,male,14.0,1366 x 768px


## Add Movement Direction

In [5]:
for datapoint in all_data:
    sequence = []
    for index, row in datapoint.iterrows():
        # print(row)
        if index % 2 == 0:
            sequence.append('l-r')
        else:
            sequence.append('r-l')
    #print(sequence)
    datapoint['Direction'] = sequence

all_data[0].head()

,part_id,width,length,ID,MT,Age,Gender,Screen Size in inch,Real Res,Direction
0,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,l-r
1,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,r-l
2,12299398,57,1207,21.175439,7171,25.0,male,14.0,1366 x 768px,l-r
3,12299398,34,905,26.617647,12359,25.0,male,14.0,1366 x 768px,r-l
4,12299398,23,302,13.130435,-1,25.0,male,14.0,1366 x 768px,l-r


# Add Display diagonal

In [7]:
import math

In [8]:
def get_diagonal(x):
    displayX = int(x.split(' x ')[0].replace('px', ''))
    displayY = int(x.split(' x ')[1].replace('px', ''))
    return math.sqrt(math.pow(displayX, 2) + math.pow(displayY, 2))

for datapoint in all_data:
    diagonal = [get_diagonal(x) for x in datapoint['Real Res']]
    datapoint['display diagonal'] = diagonal

all_data[0]

,part_id,width,length,ID,MT,Age,Gender,Screen Size in inch,Real Res,Direction,display diagonal
0,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,l-r,1567.09285
1,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,r-l,1567.09285
2,12299398,57,1207,21.175439,7171,25.0,male,14.0,1366 x 768px,l-r,1567.09285
3,12299398,34,905,26.617647,12359,25.0,male,14.0,1366 x 768px,r-l,1567.09285
4,12299398,23,302,13.130435,-1,25.0,male,14.0,1366 x 768px,l-r,1567.09285
...,...,...,...,...,...,...,...,...,...,...,...
202,12299398,102,905,8.872549,768,25.0,male,14.0,1366 x 768px,l-r,1567.09285
203,12299398,68,1207,17.750000,1834,25.0,male,14.0,1366 x 768px,r-l,1567.09285
204,12299398,90,1207,13.411111,1468,25.0,male,14.0,1366 x 768px,l-r,1567.09285
205,12299398,90,302,3.355556,255,25.0,male,14.0,1366 x 768px,r-l,1567.09285


# Prepare MT Analysis Excel

One dataframe/file for all participants, one row per trial, Errortrials are removed.

## Combine dataframes

In [9]:
import copy

In [10]:
frames = []

for datapoint in all_data:
    curr = copy.deepcopy(datapoint)
    frames.append(curr)

combined = pd.concat(frames)

combined

,part_id,width,length,ID,MT,Age,Gender,Screen Size in inch,Real Res,Direction,display diagonal
0,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,l-r,1567.09285
1,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,r-l,1567.09285
2,12299398,57,1207,21.175439,7171,25.0,male,14.0,1366 x 768px,l-r,1567.09285
3,12299398,34,905,26.617647,12359,25.0,male,14.0,1366 x 768px,r-l,1567.09285
4,12299398,23,302,13.130435,-1,25.0,male,14.0,1366 x 768px,l-r,1567.09285
...,...,...,...,...,...,...,...,...,...,...,...
229,99307457,23,302,13.130435,-1,23.0,female,26.0,1366 x 768px,r-l,1567.09285
230,99307457,23,302,13.130435,-1,23.0,female,26.0,1366 x 768px,l-r,1567.09285
231,99307457,23,302,13.130435,5061,23.0,female,26.0,1366 x 768px,r-l,1567.09285
232,99307457,23,302,13.130435,4436,23.0,female,26.0,1366 x 768px,l-r,1567.09285


## Table 1: ID vs MT

In [25]:
individual_part_ids = combined['part_id'].unique()
individual_part_ids

array(['12299398', '14239653', '15065059', '17722961', '17978640',
       '17985804', '19330011', '20073570', '21245612', '2183014',
       '23977713', '26401324', '26712442', '26886273', '27867816',
       '28184233', '28315891', '28725313', '2881314', '30669565',
       '31136533', '31256077', '33744500', '33848466', '34198438',
       '34547995', '39618267', '40159356', '41096354', '44264673',
       '47459713', '50393653', '50400149', '50866194', '54567679',
       '58472494', '60997059', '66283818', '70165854', '70882663',
       '71686964', '72852612', '73406414', '73616785', '76584802',
       '77439901', '82493269', '82493972', '82656133', '84736176',
       '86821191', '88009054', '88319338', '89184070', '89532417',
       '94667625', '95528384', '97962051', '98975745', '99307457'],
      dtype=object)

In [26]:
individual_ids = combined['ID'].unique()
individual_ids

array([21.1754386 , 26.61764706, 13.13043478,  6.71111111,  3.82278481,
        7.63291139, 10.05555556,  2.96078431, 17.75      , 11.83333333,
       52.47826087, 13.4       , 11.4556962 ,  8.86764706, 15.27848101,
       35.5       , 20.11111111,  8.88235294, 15.87719298,  8.87254902,
        4.44117647, 13.30882353,  6.7       , 39.34782609, 10.57894737,
       17.73529412, 26.82222222, 13.41111111,  3.35555556, 26.2173913 ,
        5.91176471,  5.29824561])

In [16]:
id_v_mt = copy.deepcopy(combined)
id_v_mt = id_v_mt[id_v_mt['MT'] != -1]
id_v_mt.drop(['width', 'length', 'Age', 'Gender', 'Screen Size in inch', 'Real Res', 'Direction', 'display diagonal'], inplace=True, axis=1)

arr = []
for pid in individual_part_ids:
    for diff in individual_ids:
        mt_per_id = id_v_mt[(id_v_mt['part_id'] == pid) & (id_v_mt['ID'] == diff)]['MT'].mean()
        arr.append([pid, diff, mt_per_id])

id_v_mt = pd.DataFrame(arr, columns=('part_id', 'ID', 'MT'))
id_v_mt

,part_id,ID,MT
0,12299398,21.175439,5207.4
1,12299398,26.617647,5434.2
2,12299398,13.130435,2842.4
3,12299398,6.711111,894.4
4,12299398,3.822785,372.2
...,...,...,...
1915,99307457,13.411111,763.2
1916,99307457,3.355556,137.8
1917,99307457,26.217391,5150.2
1918,99307457,5.911765,293.6


## Table 2: ID vs Errorrate

In [17]:
error_trials = combined[(combined['part_id'] == individual_part_ids[0]) & (combined['MT'] == -1)]
error_trials[error_trials['ID'] == individual_ids[15]]['MT'].count()

3

In [23]:
combined.head()

,part_id,width,length,ID,MT,Age,Gender,Screen Size in inch,Real Res,Direction,display diagonal
0,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,l-r,1567.09285
1,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,r-l,1567.09285
2,12299398,57,1207,21.175439,7171,25.0,male,14.0,1366 x 768px,l-r,1567.09285
3,12299398,34,905,26.617647,12359,25.0,male,14.0,1366 x 768px,r-l,1567.09285
4,12299398,23,302,13.130435,-1,25.0,male,14.0,1366 x 768px,l-r,1567.09285


In [28]:
arr = []
for i in range(individual_part_ids.shape[0]):
    error_trials = combined[(combined['part_id'] == individual_part_ids[i]) & (combined['MT'] == -1)]
    for j in range(individual_ids.shape[0]):
        errors = error_trials[error_trials['ID'] == individual_ids[j]]['MT'].count() / combined[(combined['part_id'] == individual_part_ids[i]) & (combined['ID'] == individual_ids[j])]['MT'].count()
        arr.append([individual_part_ids[i], individual_ids[j], errors])

In [30]:
id_v_error = pd.DataFrame(arr, columns=('part_id', 'ID', 'Error Rate'))
id_v_error

,part_id,ID,Error Rate
0,12299398,21.175439,0.375000
1,12299398,26.617647,0.166667
2,12299398,13.130435,0.545455
3,12299398,6.711111,0.166667
4,12299398,3.822785,0.000000
...,...,...,...
1915,99307457,13.411111,0.000000
1916,99307457,3.355556,0.166667
1917,99307457,26.217391,0.375000
1918,99307457,5.911765,0.000000


In [33]:
print(f'Mean Error Rate: {id_v_error["Error Rate"].mean()} -- SD: {id_v_error["Error Rate"].std()}')

Mean Error Rate: 0.15965295689967127 -- SD: 0.19977516333049417


## Table 3: ID vs Mean Errorate per ID

In [32]:
arr = []
for i in range(individual_ids.shape[0]):
    mean_error_rate = id_v_error[id_v_error['ID'] == individual_ids[i]]['Error Rate'].mean()
    std_error_rate = id_v_error[id_v_error['ID'] == individual_ids[i]]['Error Rate'].std()
    arr.append([individual_ids[i], mean_error_rate, std_error_rate])

id_v_mean_error = pd.DataFrame(arr, columns=('ID', 'Mean Errors', 'SD'))
id_v_mean_error

,ID,Mean Errors,SD
0,21.175439,0.213158,0.179659
1,26.617647,0.292554,0.209296
2,13.130435,0.254955,0.181881
3,6.711111,0.108366,0.130853
4,3.822785,0.049603,0.090615
5,7.632911,0.058631,0.117636
6,10.055556,0.075298,0.103086
7,2.960784,0.013095,0.051202
8,17.750000,0.157849,0.184722
9,11.833333,0.099336,0.128852


In [40]:
id_v_mean_error.sort_values('ID')

,ID,Mean Errors,SD
7,2.960784,0.013095,0.051202
28,3.355556,0.033234,0.080065
4,3.822785,0.049603,0.090615
20,4.441176,0.035218,0.084764
31,5.298246,0.066634,0.125403
30,5.911765,0.073082,0.115869
22,6.700000,0.075165,0.110184
3,6.711111,0.108366,0.130853
5,7.632911,0.058631,0.117636
13,8.867647,0.087500,0.128659


In [34]:
id_v_mean_error[id_v_mean_error['Mean Errors'] == id_v_mean_error['Mean Errors'].min()]

,ID,Mean Errors,SD
7,2.960784,0.013095,0.051202


In [35]:
id_v_mean_error[id_v_mean_error['Mean Errors'] == id_v_mean_error['Mean Errors'].max()]

,ID,Mean Errors,SD
10,52.478261,0.527404,0.231468


In [38]:
id_v_mean_error.to_csv(base_path + '/mean_error_rate_per_id.csv', sep=';')

## Table 4: Throughput per participant v direction

Throughput = Mean(MT/ID)

In [22]:
tp_v_direction = copy.deepcopy(combined)

tp_v_direction.drop(['width', 'length', 'Age', 'Gender', 'Screen Size in inch', 'Real Res', 'display diagonal'], inplace=True, axis=1)

tp_v_direction = tp_v_direction[tp_v_direction['MT'] != -1]

tp_v_direction

,part_id,ID,MT,Direction
2,12299398,21.175439,7171,l-r
3,12299398,26.617647,12359,r-l
7,12299398,13.130435,4007,r-l
8,12299398,6.711111,1533,l-r
9,12299398,3.822785,542,r-l
...,...,...,...,...
227,99307457,39.347826,21220,r-l
228,99307457,5.911765,188,l-r
231,99307457,13.130435,5061,r-l
232,99307457,13.130435,4436,l-r


In [23]:
l_n_r = tp_v_direction[tp_v_direction['Direction'] == 'l-r']
r_n_l = tp_v_direction[tp_v_direction['Direction'] == 'r-l']

In [24]:
arr = []
for pid in individual_part_ids:
    throughput_l = (l_n_r[l_n_r['part_id'] == pid]['ID'] / (l_n_r[l_n_r['part_id'] == pid]['MT'] / 1000)).sum() / l_n_r[l_n_r['part_id'] == pid]['MT'].count()
    throughput_r = (r_n_l[r_n_l['part_id'] == pid]['ID'] / (r_n_l[r_n_l['part_id'] == pid]['MT'] / 1000)).sum() / r_n_l[r_n_l['part_id'] == pid]['MT'].count()
    arr.append([pid, throughput_l, 'l-r'])
    arr.append([pid, throughput_r, 'r-l'])

tp_v_direction = pd.DataFrame(arr, columns=('part_id', 'TP', 'Direction'))
tp_v_direction

,part_id,TP,Direction
0,12299398,7.738337,l-r
1,12299398,8.562238,r-l
2,14239653,10.825069,l-r
3,14239653,13.984105,r-l
4,15065059,45.016550,l-r
...,...,...,...
115,97962051,53.334343,r-l
116,98975745,5.789354,l-r
117,98975745,7.540131,r-l
118,99307457,16.840208,l-r


## Table 5: Throughput per participant vs Diagonal vs Screensize

In [25]:
tp_v_screen = copy.deepcopy(combined)

tp_v_screen.drop(['width', 'length', 'Age', 'Gender', 'Real Res', 'Direction'], inplace=True, axis=1)

tp_v_screen = tp_v_screen[tp_v_screen['MT'] != -1]

tp_v_screen

,part_id,ID,MT,Screen Size in inch,display diagonal
2,12299398,21.175439,7171,14.0,1567.09285
3,12299398,26.617647,12359,14.0,1567.09285
7,12299398,13.130435,4007,14.0,1567.09285
8,12299398,6.711111,1533,14.0,1567.09285
9,12299398,3.822785,542,14.0,1567.09285
...,...,...,...,...,...
227,99307457,39.347826,21220,26.0,1567.09285
228,99307457,5.911765,188,26.0,1567.09285
231,99307457,13.130435,5061,26.0,1567.09285
232,99307457,13.130435,4436,26.0,1567.09285


In [26]:
arr = []
for pid in individual_part_ids:
    throughput = (tp_v_screen[tp_v_screen['part_id'] == pid]['ID'] / (tp_v_screen[tp_v_screen['part_id'] == pid]['MT'] / 1000)).sum() / tp_v_screen[tp_v_screen['part_id'] == pid]['MT'].count()

    arr.append([pid, throughput, tp_v_screen[tp_v_screen['part_id'] == pid]['Screen Size in inch'].iloc[0], tp_v_screen[tp_v_screen['part_id'] == pid]['display diagonal'].iloc[0]])

tp_v_screen = pd.DataFrame(arr, columns=('part_id', 'throughput', 'screen size', 'diagonal screen res'))
tp_v_screen

,part_id,throughput,screen size,diagonal screen res
0,12299398,8.124540,14.000,1567.092850
1,14239653,12.424331,27.000,2202.907170
2,15065059,47.976523,12.000,1280.000000
3,17722961,14.519404,14.000,1567.092850
4,17978640,21.008840,23.000,1561.865551
5,17985804,10.814512,21.500,2202.907170
6,19330011,4.560821,24.000,1567.092850
7,20073570,4.102584,19.000,1567.092850
8,21245612,198.789772,24.000,1567.092850
9,2183014,20.821150,15.000,1567.092850


## Table 6: ID vs Mean Movement Time per ID

In [27]:
id_v_mean_mt = copy.deepcopy(combined)

id_v_mean_mt.drop(['part_id', 'width', 'length', 'Age', 'Gender', 'Real Res', 'Direction', 'Screen Size in inch', 'display diagonal'], inplace=True, axis=1)

id_v_mean_mt = id_v_mean_mt[id_v_mean_mt['MT'] != -1]

id_v_mean_mt

,ID,MT
2,21.175439,7171
3,26.617647,12359
7,13.130435,4007
8,6.711111,1533
9,3.822785,542
...,...,...
227,39.347826,21220
228,5.911765,188
231,13.130435,5061
232,13.130435,4436


In [28]:
arr = []
for curr_id in individual_ids:
    mean_mt = id_v_mean_mt[id_v_mean_mt['ID'] == curr_id]['MT'].mean()
    arr.append([curr_id, mean_mt])

id_v_mean_mt = pd.DataFrame(arr, columns=('ID', 'Mean MT'))

id_v_mean_mt

,ID,Mean MT
0,21.175439,2553.140000
1,26.617647,3252.033445
2,13.130435,1636.615385
3,6.711111,630.993333
4,3.822785,392.410000
5,7.632911,780.528428
6,10.055556,1112.765886
7,2.960784,342.256667
8,17.750000,2118.650000
9,11.833333,1538.156667


# Export zu Excel

In [29]:
os.getcwd()

'C:\\Users\\ragor\\OneDrive\\Dokumente\\Bachelorarbeit\\BA-Prediction-Beyond-the-Systems-Latency\\01_Data Collection\\02_Data\\MT Analysis\\demographic'

In [30]:
os.chdir('..')

writer = pd.ExcelWriter('all_data2.xlsx', engine='xlsxwriter')
id_v_mt.to_excel(writer, sheet_name='ID v MT')
id_v_error.to_excel(writer, sheet_name='ID v Error')
id_v_mean_error.to_excel(writer, sheet_name='ID v Mean Error')
tp_v_direction.to_excel(writer, sheet_name='TP v Direction')
tp_v_screen.to_excel(writer, sheet_name='TP v Screen')
id_v_mean_mt.to_excel(writer, sheet_name='TP v Mean MT')

writer.save()

In [31]:
writer.close()

C:\Users\ragor\anaconda3\envs\Comp Intelligence\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


# Prepare Error rate Analysis Excel

One row per participant per ID, Count errors on this ID and divide by count+5 -> errorrate per ID

In [67]:
all_data[0]

,part_id,width,length,ID,MT,Age,Gender,Screen Size in inch,Real Res,Direction
0,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,l-r
1,12299398,57,1207,21.175439,-1,25.0,male,14.0,1366 x 768px,r-l
2,12299398,57,1207,21.175439,7171,25.0,male,14.0,1366 x 768px,l-r
3,12299398,34,905,26.617647,12359,25.0,male,14.0,1366 x 768px,r-l
4,12299398,23,302,13.130435,-1,25.0,male,14.0,1366 x 768px,l-r
...,...,...,...,...,...,...,...,...,...,...
202,12299398,102,905,8.872549,768,25.0,male,14.0,1366 x 768px,l-r
203,12299398,68,1207,17.750000,1834,25.0,male,14.0,1366 x 768px,r-l
204,12299398,90,1207,13.411111,1468,25.0,male,14.0,1366 x 768px,l-r
205,12299398,90,302,3.355556,255,25.0,male,14.0,1366 x 768px,r-l


In [79]:
removed = []
errordfs = []

one_data = [all_data[0]]

for datapoint in one_data:
    curr = copy.deepcopy(datapoint)

    individual_ids = set()

    for index, row in datapoint.iterrows():
        individual_ids.add(row['ID'])

    """
    for iid in individual_ids:
        errorcount = 0
        id_error_dict = dict()
        for index, row in datapoint.iterrows():
            if row['ID'] == iid:
                if row['MT'] == -1:
                    errorcount += 1
            id_error_dict[row['ID']] = errorcount
            # print(id_error_dict)
        errordfs.append(pd.DataFrame.from_records([id_error_dict]))
    """

    errors_for_part = []
    for iid in individual_ids:
        curr_iid = datapoint[datapoint['ID'] == iid]
        # print(curr_id)
        error_count = len(curr_iid[curr_iid['MT'] == -1].index)

        errdf = {
            'part_id': datapoint['part_id'].values[0],
            'ID': curr_iid['ID'].values[0],
            'Errors': error_count
        }

        #print(errdf)
        errors_for_part.append(errdf)

    errordfs.append(pd.DataFrame.from_records(errors_for_part))

    # print(individual_ids)
    removed.append(curr)

errordfs[0]

,part_id,ID,Errors
0,12299398,2.960784,0
1,12299398,3.822785,0
2,12299398,4.441176,0
3,12299398,3.355556,0
4,12299398,6.711111,1
5,12299398,7.632911,0
6,12299398,8.867647,1
7,12299398,8.882353,0
8,12299398,10.055556,0
9,12299398,11.833333,1


# Throughput

In [11]:
tp_data = copy.deepcopy(combined)

In [18]:
tp_data = tp_data[tp_data['MT'] != -1]
tp_data['MT'] = tp_data['MT']/1000
tp_data.head()

,part_id,width,length,ID,MT,Age,Gender,Screen Size in inch,Real Res,Direction,display diagonal
2,12299398,57,1207,21.175439,7.171,25.0,male,14.0,1366 x 768px,l-r,1567.09285
3,12299398,34,905,26.617647,12.359,25.0,male,14.0,1366 x 768px,r-l,1567.09285
7,12299398,23,302,13.130435,4.007,25.0,male,14.0,1366 x 768px,r-l,1567.09285
8,12299398,45,302,6.711111,1.533,25.0,male,14.0,1366 x 768px,l-r,1567.09285
9,12299398,79,302,3.822785,0.542,25.0,male,14.0,1366 x 768px,r-l,1567.09285


In [19]:
tp_data['part_id'] = tp_data['part_id'].astype(int)
unique_part = tp_data['part_id'].unique()
unique_part

array([12299398, 14239653, 15065059, 17722961, 17978640, 17985804,
       19330011, 20073570, 21245612,  2183014, 23977713, 26401324,
       26712442, 26886273, 27867816, 28184233, 28315891, 28725313,
        2881314, 30669565, 31136533, 31256077, 33744500, 33848466,
       34198438, 34547995, 39618267, 40159356, 41096354, 44264673,
       47459713, 50393653, 50400149, 50866194, 54567679, 58472494,
       60997059, 66283818, 70165854, 70882663, 71686964, 72852612,
       73406414, 73616785, 76584802, 77439901, 82493269, 82493972,
       82656133, 84736176, 86821191, 88009054, 88319338, 89184070,
       89532417, 94667625, 95528384, 97962051, 98975745, 99307457])

In [20]:
tp_per_part = []

for curr_part in unique_part:
    curr_data = tp_data[tp_data['part_id'] == curr_part]
    curr_tp = (curr_data['ID'] / curr_data['MT']).mean()
    tp_per_part.append([curr_part, curr_tp])

tp_per_part = pd.DataFrame(tp_per_part, columns=('part_id', 'TP'))
tp_per_part

,part_id,TP
0,12299398,8.124540
1,14239653,12.424331
2,15065059,47.976523
3,17722961,14.519404
4,17978640,21.008840
5,17985804,10.814512
6,19330011,4.560821
7,20073570,4.102584
8,21245612,198.789772
9,2183014,20.821150


In [22]:
print(f'Mean TP: {tp_per_part["TP"].mean()} -- SD TP: {tp_per_part["TP"].std()}')

Mean TP: 26.606342703112194 -- SD TP: 34.28935643525853
